## Load the Dataset

In [2]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
df.shape

(569, 33)

In [6]:
df.drop(['id', 'Unnamed: 32'], axis=1, inplace=True)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
df.shape

(569, 31)

## Basic Preprocessing

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('diagnosis', axis=1), df['diagnosis'], test_size=0.2, random_state=42)

### StandardScaler

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
X_test

array([[-0.46649743, -0.13728933, -0.44421138, ..., -0.19435087,
         0.17275669,  0.20372995],
       [ 1.36536344,  0.49866473,  1.30551088, ...,  0.99177862,
        -0.561211  , -1.00838949],
       [ 0.38006578,  0.06921974,  0.40410139, ...,  0.57035018,
        -0.10783139, -0.20629287],
       ...,
       [-0.73547237, -0.99852603, -0.74138839, ..., -0.27741059,
        -0.3820785 , -0.32408328],
       [ 0.02898271,  2.0334026 ,  0.0274851 , ..., -0.49027026,
        -1.60905688, -0.33137507],
       [ 1.87216885,  2.80077153,  1.80354992, ...,  0.7925579 ,
        -0.05868885, -0.09467243]])

### LabelEncoder

In [11]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [12]:
y_test

array([0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1])

### NumPy array to Tensor

In [13]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [15]:
X_train_tensor.shape

torch.Size([455, 30])

## Training Process

In [78]:
class SimpleNN():

    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
        self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)

    def forward(self, X):
        z = torch.matmul(X, self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred

    def loss_function(self, y, y_pred):
        # Clamp predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)

        # Calculate Loss
        loss = -(y * torch.log(y_pred) + (1-y) * torch.log(1 - y_pred)).mean()
        return loss

    def back_prop(self, loss):
        loss.backward()

    def update_parameters(self, learning_rate):
        with torch.no_grad():
            self.weights -= learning_rate * self.weights.grad
            self.bias -= learning_rate * self.bias.grad
            self.weights.grad.zero_()
            self.bias.grad.zero_()

    def evaluate_model(self, x_test, y_test):
        with torch.no_grad():
            y_pred = self.forward(x_test)
            y_pred = (y_pred > 0.5).float()
            accuracy = (y_pred == y_test).float().mean()
            return accuracy

In [86]:
# Important Parameters
learning_rate = 0.1
epochs = 25

In [87]:
# Training Pipeline
model = SimpleNN(X_train_tensor)

for epoch in range(epochs):
    y_pred = model.forward(X_train_tensor)
    loss = model.loss_function(y_test_tensor, y_pred)

    model.back_prop(loss)
    model.update_parameters(learning_rate)

    acc = model.evaluate_model(X_test_tensor, y_test_tensor)

    print(f"Epoch: {epoch+1}, Loss : {loss.item():.4f}, Accuracy : {acc.item():.2f}")


Epoch: 1, Loss : 3.7033, Accuracy : 0.52
Epoch: 2, Loss : 3.5760, Accuracy : 0.52
Epoch: 3, Loss : 3.4472, Accuracy : 0.52
Epoch: 4, Loss : 3.3151, Accuracy : 0.52
Epoch: 5, Loss : 3.1760, Accuracy : 0.52
Epoch: 6, Loss : 3.0294, Accuracy : 0.52
Epoch: 7, Loss : 2.8808, Accuracy : 0.52
Epoch: 8, Loss : 2.7322, Accuracy : 0.52
Epoch: 9, Loss : 2.5768, Accuracy : 0.52
Epoch: 10, Loss : 2.4195, Accuracy : 0.52
Epoch: 11, Loss : 2.2614, Accuracy : 0.52
Epoch: 12, Loss : 2.1019, Accuracy : 0.51
Epoch: 13, Loss : 1.9374, Accuracy : 0.51
Epoch: 14, Loss : 1.7751, Accuracy : 0.52
Epoch: 15, Loss : 1.6192, Accuracy : 0.52
Epoch: 16, Loss : 1.4738, Accuracy : 0.52
Epoch: 17, Loss : 1.3378, Accuracy : 0.51
Epoch: 18, Loss : 1.2172, Accuracy : 0.52
Epoch: 19, Loss : 1.1132, Accuracy : 0.51
Epoch: 20, Loss : 1.0264, Accuracy : 0.52
Epoch: 21, Loss : 0.9565, Accuracy : 0.51
Epoch: 22, Loss : 0.9019, Accuracy : 0.52
Epoch: 23, Loss : 0.8602, Accuracy : 0.52
Epoch: 24, Loss : 0.8285, Accuracy : 0.53
E

In [70]:
model.weights

tensor([[ 0.2771],
        [-0.0178],
        [ 0.3675],
        [ 0.3492],
        [ 0.1128],
        [-0.4620],
        [ 0.1817],
        [ 0.2048],
        [-0.0570],
        [-0.0107],
        [-0.0646],
        [ 0.1321],
        [ 0.3610],
        [-0.0547],
        [ 0.5869],
        [ 0.0189],
        [-0.0416],
        [-0.0681],
        [ 0.1937],
        [ 0.3076],
        [ 0.3031],
        [ 0.1461],
        [-0.4445],
        [-0.2531],
        [ 0.1843],
        [-0.5540],
        [-0.4049],
        [ 0.2952],
        [-0.1262],
        [ 0.4110]], dtype=torch.float64, requires_grad=True)

In [71]:
model.bias

tensor([-0.0996], dtype=torch.float64, requires_grad=True)